<a href="https://colab.research.google.com/github/AsliDK/Final-Year-Project/blob/main/fyp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Task 1: Environment Setup

Install Python: Ensure you have Python installed on your system.
Create a Virtual Environment: Use virtualenv to create an isolated environment for your project dependencies.
Install Required Libraries: Install Flask or Django, Tweepy, Pandas, GeoPandas, Scikit-learn, and any other necessary libraries using pip.

Task 2: Backend Development

Initialize Flask/Django Project: Create a new Flask or Django project as your application's backend framework.
Twitter API Integration:
Register your application with Twitter to obtain API keys.
Use Tweepy in a Python script to fetch geo-tagged tweets based on your criteria (location, keywords).

Data Processing:
Write Python scripts to clean and preprocess the data using Pandas.
Use GeoPandas for any spatial data operations needed.

Analysis & Machine Learning:
Implement algorithms to analyze patterns, preferences, and peak times in the collected data.
Use Scikit-learn for any machine learning models.

Task 3: Frontend Development

Basic Web Layout:
Create HTML templates for your main page, including forms for inputs (location, time frame, activity type).
Style your templates with CSS.

JavaScript and Map Integration:
Integrate Leaflet.js or Google Maps API to display an interactive map.
Use JavaScript to handle user inputs (forms) and interact with your backend for data retrieval.

Task 4: Connecting Backend and Frontend

API Development:
Develop RESTful APIs in your Flask/Django application to serve processed data to the frontend.

AJAX Requests:
Use AJAX in your JavaScript code to fetch data from your backend APIs and update the frontend dynamically.

Task 5: Data Visualization Integration
Integrate Plotly.js or D3.js
Add interactive charts and graphs to your frontend for displaying analysis results.

Task 6: Testing and Refinement
Unit Testing:
Write tests for both backend and frontend functionalities to ensure reliability.
System Integration Testing:
Test the entire system end-to-end, from data collection to visualization on the frontend.

Task 7: Deployment
Choose a Hosting Service: Decide on a cloud platform like Heroku, AWS, or Google Cloud.
Deploy Your Application: Follow the hosting service's guide to deploy your Flask/Django application.

Task 8: Documentation
Document Your Code: Ensure all your code is well-commented and documented.
User Guide: Write a user guide explaining how to use the system


Example Scenario:
A user selects "Soho, London" as the area of interest chooses "July 2023" as the time frame, and filters for "dining and nightlife" activities.

The system processes geo-tagged tweets from July 2023 related to dining and nightlife in Soho, identifies leisure hubs, and displays them on the map.

The map shows markers where activity is highest. Clicking on a marker displays detailed info such as popular venues, peak activity times, and possibly user sentiments or reviews extracted from the tweets.

The user can then generate a report summarising the findings, including recommendations for urban planning or business development in Soho.
This design bridges the gap between complex data analysis and user-friendly interaction, making urban planning insights accessible to a wider audience.


In [ ]:
import requests

url = "https://api.foursquare.com/v3/places/search"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3PR9pOx+9xtJcWbhH2KCcLXqG7c83PTpavbRqXocj1Jc="
}

params = {
    "near": "London, UK",
    "limit": 50  # 100 mekan bilgisi almak için limiti belirle
}

response = requests.get(url, headers=headers, params=params)

print(response.text)


{"results":[{"fsq_id":"4ac518cdf964a520e6a520e3","categories":[{"id":10004,"name":"Art Gallery","short_name":"Art Gallery","plural_name":"Art Galleries","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/arts_entertainment/artgallery_","suffix":".png"}}],"chains":[],"closed_bucket":"VeryLikelyOpen","distance":193,"geocodes":{"main":{"latitude":51.508957,"longitude":-0.128678},"roof":{"latitude":51.508742,"longitude":-0.128459}},"link":"/v3/places/4ac518cdf964a520e6a520e3","location":{"address":"Trafalgar Sq","admin_region":"England","country":"GB","cross_street":"","formatted_address":"Trafalgar Sq, London, Greater London, WC2N 5DN","locality":"London","post_town":"London","postcode":"WC2N 5DN","region":"Greater London"},"name":"National Gallery","related_places":{"children":[{"fsq_id":"4cd183159d87224b4c405b3b","categories":[{"id":13035,"name":"Coffee Shop","short_name":"Coffee Shop","plural_name":"Coffee Shops","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/coffe